In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
!bash download_voc.sh

--2021-02-21 14:13:07--  http://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz
Resolving www.eecs.berkeley.edu (www.eecs.berkeley.edu)... 23.185.0.1, 2620:12a:8000::1, 2620:12a:8001::1
Connecting to www.eecs.berkeley.edu (www.eecs.berkeley.edu)|23.185.0.1|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz [following]
--2021-02-21 14:13:07--  https://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz
Connecting to www.eecs.berkeley.edu (www.eecs.berkeley.edu)|23.185.0.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz [following]
--2021-02-21 14:13:07--  https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic

In [ ]:
!wget 'https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/extract_weights.py'
!wget 'https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/load_weights.py'
!wget 'https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/model.py'

--2021-02-21 14:22:12--  https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/extract_weights.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3648 (3.6K) [text/plain]
Saving to: ‘extract_weights.py’

extract_weights.py  100%[===================>]   3.56K  --.-KB/s    in 0s      

2021-02-21 14:22:13 (77.5 MB/s) - ‘extract_weights.py’ saved [3648/3648]

--2021-02-21 14:22:13--  https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/load_weights.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

In [ ]:
from VOCGen import get_voc_datagen
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LambdaCallback
from tensorflow.keras.losses import categorical_crossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Mean, BinaryAccuracy
from tensorflow.keras.losses import BinaryCrossentropy
from gan_model import AdverGAN
import os
from metrics import mean_iou

In [ ]:
# 20 classes from PASCAL VOC and 1 background
N_CLASS = 20 + 1 
BATCH_SIZE = 12
img_target_size = (320, 320)

In [ ]:
img_preproc = lambda x: x / 255.
tr_gen, val_gen = get_voc_datagen(N_CLASS, BATCH_SIZE, img_preproc, None, img_target_size)

In [ ]:
g_iou_tracker = Mean(name='iou')
gan_acc_tracker = Mean(name='gan_acc')
disc_acc_tracker = Mean(name='disc_acc')
iou_score = mean_iou
binary_entropy = BinaryCrossentropy()
gan_acc = BinaryAccuracy()
disc_acc = BinaryAccuracy()

In [ ]:
for x,y in tr_gen:
  print(x.shape)
  print(y.shape)
  break

(12, 320, 320, 3)
(12, 320, 320, 21)


In [ ]:
gan_model = AdverGAN(1e-2, img_target_size, N_CLASS)
g_opt = Adam(2.5e-4)
d_opt = Adam(1e-5)

166100992/166096424 [==============================] - 3s 0us/step


In [ ]:
BEST_VAL_IOU = 0.0
MODEL_SAVE_DIR = os.path.join('drive', 'MyDrive', 'adverseg_models')

def save_generator_discriminator_on_best(logs):
  global BEST_VAL_IOU
  if BEST_VAL_IOU < logs['val_gen_iou']:
    BEST_VAL_IOU = logs['val_gen_iou']
    gan_model.save_models(MODEL_SAVE_DIR)


saving_model_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: save_generator_discriminator_on_best(logs)
)

In [ ]:
gan_model.compile(g_opt, d_opt, binary_entropy, gan_acc, disc_acc, categorical_crossentropy, iou_score, g_iou_tracker, gan_acc_tracker, disc_acc_tracker)

In [ ]:
gan_model.fit(tr_gen, epochs=40, validation_data=val_gen, callbacks=[saving_model_callback])

Epoch 1/40
709/709 [==============================] - 1252s 2s/step - disc_fake_loss: 0.6909 - disc_real_loss: 0.6103 - gan_loss: 0.4242 - gen_iou: 0.5719 - g_loss: 0.4166 - disc_acc: 0.5559 - gan_acc: 0.5726 - val_disc_fake_loss: 0.8329 - val_disc_real_loss: 0.5240 - val_gan_loss: 0.0642 - val_gen_iou: 0.5757 - val_g_loss: 0.0584 - val_disc_acc: 0.6515 - val_gan_acc: 0.4562
Epoch 2/40
709/709 [==============================] - 1238s 2s/step - disc_fake_loss: 0.5019 - disc_real_loss: 0.4166 - gan_loss: 0.3119 - gen_iou: 0.6457 - g_loss: 0.2922 - disc_acc: 0.7065 - gan_acc: 0.4261 - val_disc_fake_loss: 0.6747 - val_disc_real_loss: 0.4355 - val_gan_loss: 0.0727 - val_gen_iou: 0.6537 - val_g_loss: 0.0633 - val_disc_acc: 0.7390 - val_gan_acc: 0.3950
Epoch 3/40
709/709 [==============================] - 1238s 2s/step - disc_fake_loss: 0.4684 - disc_real_loss: 0.3829 - gan_loss: 0.2720 - gen_iou: 0.6941 - g_loss: 0.2449 - disc_acc: 0.7563 - gan_acc: 0.3783 - val_disc_fake_loss: 0.4784 - val_